In [ ]:
pip install requests

In [ ]:
import requests

# Step 1: Define the API endpoint
url = "https://fakestoreapi.com/products"

# Step 2: Send GET request to fetch all products
response = requests.get(url)

# Step 3: Check if the request was successful
if response.status_code == 200:
    products = response.json()

    # Step 4: Loop through and display products
    for product in products:
        print("🛒 Title:", product["title"])
        print("💲 Price:", product["price"])
        print("📦 Category:", product["category"])
        print("📝 Description:", product["description"][:100], "...")
        print("⭐ Rating:", product["rating"]["rate"], "from", product["rating"]["count"], "reviews")
        print("-" * 60)

else:
    print("Failed to retrieve products. Status Code:", response.status_code)


🛒 Title: Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops
💲 Price: 109.95
📦 Category: men's clothing
📝 Description: Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in t ...
⭐ Rating: 3.9 from 120 reviews
------------------------------------------------------------
🛒 Title: Mens Casual Premium Slim Fit T-Shirts 
💲 Price: 22.3
📦 Category: men's clothing
📝 Description: Slim-fitting style, contrast raglan long sleeve, three-button henley placket, light weight & soft fa ...
⭐ Rating: 4.1 from 259 reviews
------------------------------------------------------------
🛒 Title: Mens Cotton Jacket
💲 Price: 55.99
📦 Category: men's clothing
📝 Description: great outerwear jackets for Spring/Autumn/Winter, suitable for many occasions, such as working, hiki ...
⭐ Rating: 4.7 from 500 reviews
------------------------------------------------------------
🛒 Title: Mens Casual Slim Fit
💲 Price: 15.99
📦 Category: men's clothing
📝 Description: The color co

SyntaxError: invalid character '🛒' (U+1F6D2) (<ipython-input-3-2a980645d73f>, line 1)

In [ ]:
import requests
from datetime import datetime

# Your Agro API key
API_KEY = "b3e05154db77180fcbe025ce79da934c"

# Expanded crop dataset
CROP_DATABASE = [
    {"name": "Tomato", "min_temp": 15, "max_temp": 30, "seasons": ["spring", "summer"], "notes": "Needs full sun, moderate water"},
    {"name": "Spinach", "min_temp": 5, "max_temp": 20, "seasons": ["winter", "fall"], "notes": "Prefers cooler weather, partial shade"},
    {"name": "Corn", "min_temp": 18, "max_temp": 35, "seasons": ["summer"], "notes": "Requires warm weather and lots of sun"},
    {"name": "Carrot", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Grows well in loose soil"},
    {"name": "Potato", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Cool weather crop, needs well-drained soil"},
    {"name": "Cucumber", "min_temp": 18, "max_temp": 30, "seasons": ["summer"], "notes": "Needs lots of water and sun"},
    {"name": "Lettuce", "min_temp": 4, "max_temp": 20, "seasons": ["spring", "fall"], "notes": "Thrives in cooler climates"},
    {"name": "Bell Pepper", "min_temp": 16, "max_temp": 30, "seasons": ["summer"], "notes": "Warm temperature lover, needs sun"},
    {"name": "Onion", "min_temp": 12, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Needs fertile, well-drained soil"},
    {"name": "Broccoli", "min_temp": 10, "max_temp": 20, "seasons": ["fall", "winter"], "notes": "Prefers cool weather"},
    {"name": "Strawberry", "min_temp": 13, "max_temp": 25, "seasons": ["spring"], "notes": "Needs well-drained soil and sun"},
    {"name": "Green Beans", "min_temp": 16, "max_temp": 30, "seasons": ["summer"], "notes": "Warm season crop, needs support"},
    {"name": "Peas", "min_temp": 7, "max_temp": 20, "seasons": ["spring", "fall"], "notes": "Cool weather, needs support"},
    {"name": "Pumpkin", "min_temp": 18, "max_temp": 30, "seasons": ["summer", "fall"], "notes": "Needs lots of space and sun"},
    {"name": "Zucchini", "min_temp": 18, "max_temp": 30, "seasons": ["summer"], "notes": "Requires warm temps and water"},
]

def get_current_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

def create_polygon(lat, lon):
    url = f"http://api.agromonitoring.com/agro/1.0/polygons?appid={API_KEY}"
    payload = {
        "name": "UserField",
        "geo_json": {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [lon, lat],
                    [lon + 0.01, lat],
                    [lon + 0.01, lat + 0.01],
                    [lon, lat + 0.01],
                    [lon, lat]
                ]]
            }
        }
    }
    response = requests.post(url, json=payload)
    if response.status_code == 201:
        return response.json().get("id")
    else:
        print("Error creating polygon:", response.text)
        return None

def get_weather(poly_id):
    url = f"http://api.agromonitoring.com/agro/1.0/weather?polyid={poly_id}&appid={API_KEY}"
    res = requests.get(url)
    if res.status_code == 200:
        data = res.json()
        temp_c = round(data['main']['temp'] - 273.15, 2)
        humidity = data['main']['humidity']
        return temp_c, humidity
    else:
        print("Error fetching weather:", res.text)
        return None, None

def recommend_crops(temp, season):
    print(f"\n📊 Based on {temp}°C and {season.capitalize()} season, suitable crops are:")
    found = False
    for crop in CROP_DATABASE:
        if crop["min_temp"] <= temp <= crop["max_temp"] and season in crop["seasons"]:
            print(f"✅ {crop['name']} — {crop['notes']}")
            found = True
    if not found:
        print("❌ No crops match these conditions. Consider greenhouse planting or adjusting timing.")

# === Main App ===
print("🪴 Welcome to the Smart Crop Recommender 🪴")

try:
    lat = float(input("Enter your latitude: "))
    lon = float(input("Enter your longitude: "))

    polygon_id = create_polygon(lat, lon)
    if not polygon_id:
        print("❌ Failed to create field. Check your location.")
    else:
        temp, humidity = get_weather(polygon_id)
        if temp is None:
            print("❌ Could not get weather data.")
        else:
            current_month = datetime.now().month
            season = get_current_season(current_month)

            print(f"\n🌍 Location: {lat}, {lon}")
            print(f"🌡️ Temperature: {temp} °C")
            print(f"💧 Humidity: {humidity}%")
            print(f"🕒 Season: {season.capitalize()}")

            recommend_crops(temp, season)

except Exception as e:
    print("⚠️ Error:", e)


🪴 Welcome to the Smart Crop Recommender 🪴
Enter your latitude: 23.6546944
Enter your longitude: 58.183995

🌍 Location: 23.6546944, 58.183995
🌡️ Temperature: 35.03 °C
💧 Humidity: 63%
🕒 Season: Spring

📊 Based on 35.03°C and Spring season, suitable crops are:
❌ No crops match these conditions. Consider greenhouse planting or adjusting timing.


In [ ]:
import tkinter as tk
from tkinter import messagebox, scrolledtext
import requests
from datetime import datetime

API_KEY = "b3e05154db77180fcbe025ce79da934c"

CROP_DATABASE = [
    {"name": "Tomato", "min_temp": 15, "max_temp": 30, "seasons": ["spring", "summer"], "notes": "Needs full sun, moderate water"},
    {"name": "Spinach", "min_temp": 5, "max_temp": 20, "seasons": ["winter", "fall"], "notes": "Prefers cooler weather, partial shade"},
    {"name": "Corn", "min_temp": 18, "max_temp": 35, "seasons": ["summer"], "notes": "Requires warm weather and lots of sun"},
    {"name": "Carrot", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Grows well in loose soil"},
    {"name": "Potato", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Cool weather crop, needs well-drained soil"},
    {"name": "Cucumber", "min_temp": 18, "max_temp": 30, "seasons": ["summer"], "notes": "Needs lots of water and sun"},
    {"name": "Lettuce", "min_temp": 4, "max_temp": 20, "seasons": ["spring", "fall"], "notes": "Thrives in cooler climates"},
    {"name": "Bell Pepper", "min_temp": 16, "max_temp": 30, "seasons": ["summer"], "notes": "Warm temperature lover, needs sun"},
    {"name": "Onion", "min_temp": 12, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Needs fertile, well-drained soil"},
    {"name": "Broccoli", "min_temp": 10, "max_temp": 20, "seasons": ["fall", "winter"], "notes": "Prefers cool weather"},
    {"name": "Strawberry", "min_temp": 13, "max_temp": 25, "seasons": ["spring"], "notes": "Needs well-drained soil and sun"},
    {"name": "Green Beans", "min_temp": 16, "max_temp": 30, "seasons": ["summer"], "notes": "Warm season crop, needs support"},
    {"name": "Peas", "min_temp": 7, "max_temp": 20, "seasons": ["spring", "fall"], "notes": "Cool weather, needs support"},
    {"name": "Pumpkin", "min_temp": 18, "max_temp": 30, "seasons": ["summer", "fall"], "notes": "Needs lots of space and sun"},
    {"name": "Zucchini", "min_temp": 18, "max_temp": 30, "seasons": ["summer"], "notes": "Requires warm temps and water"},
]

def get_current_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

def create_polygon(lat, lon):
    url = f"http://api.agromonitoring.com/agro/1.0/polygons?appid={API_KEY}"
    payload = {
        "name": "UserField",
        "geo_json": {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [lon, lat],
                    [lon + 0.01, lat],
                    [lon + 0.01, lat + 0.01],
                    [lon, lat + 0.01],
                    [lon, lat]
                ]]
            }
        }
    }
    response = requests.post(url, json=payload)
    if response.status_code == 201:
        return response.json().get("id")
    else:
        return None

def get_weather(poly_id):
    url = f"http://api.agromonitoring.com/agro/1.0/weather?polyid={poly_id}&appid={API_KEY}"
    res = requests.get(url)
    if res.status_code == 200:
        data = res.json()
        temp_c = round(data['main']['temp'] - 273.15, 2)
        humidity = data['main']['humidity']
        return temp_c, humidity
    else:
        return None, None

def recommend_crops(temp, season):
    matches = []
    for crop in CROP_DATABASE:
        if crop["min_temp"] <= temp <= crop["max_temp"] and season in crop["seasons"]:
            matches.append(f"{crop['name']} — {crop['notes']}")
    return matches

def on_recommend():
    try:
        lat = float(entry_lat.get())
        lon = float(entry_lon.get())
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numeric values for latitude and longitude.")
        return

    btn_recommend.config(state="disabled")
    text_result.delete("1.0", tk.END)
    text_result.insert(tk.END, "⏳ Fetching data, please wait...\n")
    root.update()

    poly_id = create_polygon(lat, lon)
    if not poly_id:
        messagebox.showerror("API Error", "Failed to create polygon with the given coordinates.")
        btn_recommend.config(state="normal")
        return

    temp, humidity = get_weather(poly_id)
    if temp is None:
        messagebox.showerror("API Error", "Failed to fetch weather data.")
        btn_recommend.config(state="normal")
        return

    month = datetime.now().month
    season = get_current_season(month)

    text_result.delete("1.0", tk.END)
    text_result.insert(tk.END, f"🌍 Location: {lat}, {lon}\n")
    text_result.insert(tk.END, f"🌡️ Temperature: {temp} °C\n")
    text_result.insert(tk.END, f"💧 Humidity: {humidity}%\n")
    text_result.insert(tk.END, f"🕒 Season: {season.capitalize()}\n\n")
    text_result.insert(tk.END, "📊 Suitable crops:\n")

    crops = recommend_crops(temp, season)
    if crops:
        for c in crops:
            text_result.insert(tk.END, f"✅ {c}\n")
    else:
        text_result.insert(tk.END, "❌ No suitable crops found for these conditions.\n")

    btn_recommend.config(state="normal")

# --- Tkinter GUI setup ---
root = tk.Tk()
root.title("Smart Crop Recommender")
root.geometry("500x500")
root.resizable(False, False)

tk.Label(root, text="Enter Latitude:").pack(pady=(10,0))
entry_lat = tk.Entry(root, width=20)
entry_lat.pack()

tk.Label(root, text="Enter Longitude:").pack(pady=(10,0))
entry_lon = tk.Entry(root, width=20)
entry_lon.pack()

btn_recommend = tk.Button(root, text="Get Recommendations", command=on_recommend)
btn_recommend.pack(pady=15)

text_result = scrolledtext.ScrolledText(root, width=60, height=20)
text_result.pack(pady=10)

root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [ ]:
import requests
from datetime import datetime

API_KEY = "b3e05154db77180fcbe025ce79da934c"

# Simple crop dataset (expandable)
CROP_DATABASE = [
    {"name": "Tomato", "min_temp": 15, "max_temp": 30, "seasons": ["spring", "summer"], "notes": "Needs full sun, moderate water"},
    {"name": "Spinach", "min_temp": 5, "max_temp": 20, "seasons": ["winter", "fall"], "notes": "Prefers cooler weather, partial shade"},
    {"name": "Corn", "min_temp": 18, "max_temp": 35, "seasons": ["summer"], "notes": "Requires warm temperatures and plenty of water"},
    {"name": "Potato", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Grows best in cool weather with rich soil"},
    {"name": "Lettuce", "min_temp": 10, "max_temp": 22, "seasons": ["spring", "fall"], "notes": "Likes mild temperatures and plenty of moisture"}
]

def get_current_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

def create_polygon(lat, lon):
    url = f"http://api.agromonitoring.com/agro/1.0/polygons?appid={API_KEY}"
    payload = {
        "name": "UserField",
        "geo_json": {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [lon, lat],
                    [lon + 0.01, lat],
                    [lon + 0.01, lat + 0.01],
                    [lon, lat + 0.01],
                    [lon, lat]
                ]]
            }
        }
    }
    response = requests.post(url, json=payload)
    if response.status_code == 201:
        return response.json().get("id")
    else:
        print("Error creating polygon:", response.text)
        return None

def get_weather(poly_id):
    url = f"http://api.agromonitoring.com/agro/1.0/weather?polyid={poly_id}&appid={API_KEY}"
    res = requests.get(url)
    if res.status_code == 200:
        data = res.json()
        temp_c = round(data['main']['temp'] - 273.15, 2)
        humidity = data['main']['humidity']
        return temp_c, humidity
    else:
        print("Error fetching weather data:", res.text)
        return None, None

def recommend_crops(temp, season):
    matches = []
    for crop in CROP_DATABASE:
        if crop["min_temp"] <= temp <= crop["max_temp"] and season in crop["seasons"]:
            matches.append(f"{crop['name']} — {crop['notes']}")
    return matches

def run_crop_recommender():
    print("Welcome to the Crop Recommender App!")

    lat = float(input("Enter your location latitude (e.g., 40.7128): "))
    lon = float(input("Enter your location longitude (e.g., -74.0060): "))

    print("Creating polygon for your location...")
    poly_id = create_polygon(lat, lon)
    if not poly_id:
        print("Failed to create polygon. Exiting.")
        return

    print("Fetching weather data...")
    temp, humidity = get_weather(poly_id)
    if temp is None:
        print("Failed to get weather data. Exiting.")
        return

    month = datetime.now().month
    season = get_current_season(month)

    print(f"\nYour Location: {lat}, {lon}")
    print(f"Temperature: {temp} °C")
    print(f"Humidity: {humidity}%")
    print(f"Season: {season.capitalize()}")

    crops = recommend_crops(temp, season)
    if crops:
        print("\nRecommended crops for your location and season:")
        for c in crops:
            print(f"- {c}")
    else:
        print("No suitable crops found for the current weather and season.")

run_crop_recommender()


Welcome to the Crop Recommender App!


KeyboardInterrupt: Interrupted by user

In [ ]:
import requests
from datetime import datetime

AGRO_API_KEY = "b3e05154db77180fcbe025ce79da934c"
GEOCODING_API = "http://api.openweathermap.org/geo/1.0/direct"
OPENWEATHERMAP_KEY = "b6907d289e10d714a6e88b30761fae22"  # Public demo key

# Expanded crop database
CROP_DATABASE = [
    {"name": "Tomato", "min_temp": 15, "max_temp": 30, "seasons": ["spring", "summer"], "notes": "Needs full sun, moderate water"},
    {"name": "Spinach", "min_temp": 5, "max_temp": 20, "seasons": ["winter", "fall"], "notes": "Prefers cooler weather, partial shade"},
    {"name": "Corn", "min_temp": 18, "max_temp": 35, "seasons": ["summer"], "notes": "Requires warm temperatures and plenty of water"},
    {"name": "Potato", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Grows best in cool weather with rich soil"},
    {"name": "Lettuce", "min_temp": 10, "max_temp": 22, "seasons": ["spring", "fall"], "notes": "Likes mild temperatures and plenty of moisture"},
    {"name": "Cabbage", "min_temp": 7, "max_temp": 24, "seasons": ["fall", "winter"], "notes": "Prefers cool weather and rich, moist soil"},
    {"name": "Carrot", "min_temp": 8, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Well-drained soil, full sun or partial shade"},
    {"name": "Okra", "min_temp": 20, "max_temp": 35, "seasons": ["summer"], "notes": "Thrives in heat and requires good drainage"},
    {"name": "Peas", "min_temp": 10, "max_temp": 24, "seasons": ["spring"], "notes": "Cool-weather crop, trellis recommended"}
]

def get_current_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

def get_coordinates(city):
    params = {"q": city, "limit": 1, "appid": OPENWEATHERMAP_KEY}
    response = requests.get(GEOCODING_API, params=params)
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return data["lat"], data["lon"]
    else:
        print("City not found or error fetching coordinates.")
        return None, None

def create_polygon(lat, lon):
    url = f"http://api.agromonitoring.com/agro/1.0/polygons?appid={AGRO_API_KEY}"
    payload = {
        "name": "UserField",
        "geo_json": {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [lon, lat],
                    [lon + 0.01, lat],
                    [lon + 0.01, lat + 0.01],
                    [lon, lat + 0.01],
                    [lon, lat]
                ]]
            }
        }
    }
    response = requests.post(url, json=payload)
    if response.status_code == 201:
        return response.json().get("id")
    else:
        print("Error creating polygon:", response.text)
        return None

def get_weather(poly_id):
    url = f"http://api.agromonitoring.com/agro/1.0/weather?polyid={poly_id}&appid={AGRO_API_KEY}"
    res = requests.get(url)
    if res.status_code == 200:
        data = res.json()
        temp_c = round(data['main']['temp'] - 273.15, 2)
        humidity = data['main']['humidity']
        return temp_c, humidity
    else:
        print("Error fetching weather data:", res.text)
        return None, None

def recommend_crops(temp, season):
    matches = []
    for crop in CROP_DATABASE:
        if crop["min_temp"] <= temp <= crop["max_temp"] and season in crop["seasons"]:
            matches.append(f"{crop['name']} — {crop['notes']}")
    return matches

def run_crop_recommender():
    print("🌱 Welcome to the Crop Recommender App!")

    city = input("Enter your city name (e.g., Muscat): ")

    lat, lon = get_coordinates(city)
    if lat is None or lon is None:
        return

    print(f"Coordinates for {city}: {lat}, {lon}")
    print("Creating polygon for your location...")

    poly_id = create_polygon(lat, lon)
    if not poly_id:
        return

    print("Fetching weather data...")
    temp, humidity = get_weather(poly_id)
    if temp is None:
        return

    season = get_current_season(datetime.now().month)

    print(f"\n📍 Location: {city}")
    print(f"🌡️ Temperature: {temp} °C")
    print(f"💧 Humidity: {humidity}%")
    print(f"🗓️ Season: {season.capitalize()}")

    crops = recommend_crops(temp, season)
    if crops:
        print("\n✅ Recommended crops for your area and season:")
        for c in crops:
            print(f"- {c}")
    else:
        print("🚫 No suitable crops found for this weather and season.")

run_crop_recommender()


🌱 Welcome to the Crop Recommender App!
Enter your city name (e.g., Muscat): Ibri
City not found or error fetching coordinates.


In [ ]:
import requests
from datetime import datetime

AGRO_API_KEY = "b3e05154db77180fcbe025ce79da934c"
GEOCODING_API = "http://api.openweathermap.org/geo/1.0/direct"
OPENWEATHERMAP_KEY = "b6907d289e10d714a6e88b30761fae22"  # Public demo key

# Expanded crop database
CROP_DATABASE = [
    {"name": "Tomato", "min_temp": 15, "max_temp": 30, "seasons": ["spring", "summer"], "notes": "Needs full sun, moderate water"},
    {"name": "Spinach", "min_temp": 5, "max_temp": 20, "seasons": ["winter", "fall"], "notes": "Prefers cooler weather, partial shade"},
    {"name": "Corn", "min_temp": 18, "max_temp": 35, "seasons": ["summer"], "notes": "Requires warm temperatures and plenty of water"},
    {"name": "Potato", "min_temp": 10, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Grows best in cool weather with rich soil"},
    {"name": "Lettuce", "min_temp": 10, "max_temp": 22, "seasons": ["spring", "fall"], "notes": "Likes mild temperatures and plenty of moisture"},
    {"name": "Cabbage", "min_temp": 7, "max_temp": 24, "seasons": ["fall", "winter"], "notes": "Prefers cool weather and rich, moist soil"},
    {"name": "Carrot", "min_temp": 8, "max_temp": 25, "seasons": ["spring", "fall"], "notes": "Well-drained soil, full sun or partial shade"},
    {"name": "Okra", "min_temp": 20, "max_temp": 35, "seasons": ["summer"], "notes": "Thrives in heat and requires good drainage"},
    {"name": "Peas", "min_temp": 10, "max_temp": 24, "seasons": ["spring"], "notes": "Cool-weather crop, trellis recommended"}
]

def get_current_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

def get_coordinates(city):
    params = {"q": city, "limit": 1, "appid": OPENWEATHERMAP_KEY}
    response = requests.get(GEOCODING_API, params=params)
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return data["lat"], data["lon"]
    else:
        print("City not found or error fetching coordinates.")
        return None, None

def create_polygon(lat, lon):
    url = f"http://api.agromonitoring.com/agro/1.0/polygons?appid={AGRO_API_KEY}"
    payload = {
        "name": "UserField",
        "geo_json": {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [lon, lat],
                    [lon + 0.01, lat],
                    [lon + 0.01, lat + 0.01],
                    [lon, lat + 0.01],
                    [lon, lat]
                ]]
            }
        }
    }
    response = requests.post(url, json=payload)
    if response.status_code == 201:
        return response.json().get("id")
    else:
        print("Error creating polygon:", response.text)
        return None

def get_weather(poly_id):
    url = f"http://api.agromonitoring.com/agro/1.0/weather?polyid={poly_id}&appid={AGRO_API_KEY}"
    res = requests.get(url)
    if res.status_code == 200:
        data = res.json()
        temp_c = round(data['main']['temp'] - 273.15, 2)
        humidity = data['main']['humidity']
        return temp_c, humidity
    else:
        print("Error fetching weather data:", res.text)
        return None, None

def recommend_crops(temp, season):
    matches = []
    for crop in CROP_DATABASE:
        if crop["min_temp"] <= temp <= crop["max_temp"] and season in crop["seasons"]:
            matches.append(f"{crop['name']} — {crop['notes']}")
    return matches

def run_crop_recommender():
    print("🌱 Welcome to the Crop Recommender App!")

    city = input("Enter your city name (e.g., Muscat): ")

    lat, lon = get_coordinates(city)
    if lat is None or lon is None:
        return

    print(f"Coordinates for {city}: {lat}, {lon}")
    print("Creating polygon for your location...")

    poly_id = create_polygon(lat, lon)
    if not poly_id:
        return

    print("Fetching weather data...")
    temp, humidity = get_weather(poly_id)
    if temp is None:
        return

    season = get_current_season(datetime.now().month)

    print(f"\n📍 Location: {city}")
    print(f"🌡️ Temperature: {temp} °C")
    print(f"💧 Humidity: {humidity}%")
    print(f"🗓️ Season: {season.capitalize()}")

    crops = recommend_crops(temp, season)
    if crops:
        print("\n✅ Recommended crops for your area and season:")
        for c in crops:
            print(f"- {c}")
    else:
        print("🚫 No suitable crops found for this weather and season.")

run_crop_recommender()


🌱 Welcome to the Crop Recommender App!
Enter your city name (e.g., Muscat): Muscat
City not found or error fetching coordinates.


In [ ]:
# STEP 1: Install required package
!pip install requests

# STEP 2: Import libraries
import requests
import pandas as pd

# STEP 3: Define functions

def get_agro_weather(lat, lon, api_key):
    """Get current weather from Agro API."""
    url = f"http://api.agromonitoring.com/agro/1.0/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to get weather data")

def recommend_crops(temp_c, humidity):
    """Return list of suitable crops based on conditions."""
    crops_data = [
        {"crop": "Tomato", "min_temp": 18, "max_temp": 27, "min_humidity": 50, "max_humidity": 70},
        {"crop": "Wheat", "min_temp": 10, "max_temp": 25, "min_humidity": 40, "max_humidity": 60},
        {"crop": "Rice", "min_temp": 20, "max_temp": 35, "min_humidity": 70, "max_humidity": 90},
        {"crop": "Potato", "min_temp": 15, "max_temp": 20, "min_humidity": 65, "max_humidity": 85},
        {"crop": "Lettuce", "min_temp": 10, "max_temp": 22, "min_humidity": 60, "max_humidity": 80},
        {"crop": "Onion", "min_temp": 12, "max_temp": 24, "min_humidity": 40, "max_humidity": 60},
        {"crop": "Corn", "min_temp": 16, "max_temp": 32, "min_humidity": 50, "max_humidity": 70},
        {"crop": "Carrot", "min_temp": 12, "max_temp": 23, "min_humidity": 60, "max_humidity": 80},
    ]
    df = pd.DataFrame(crops_data)
    suitable = df[
        (df['min_temp'] <= temp_c) & (df['max_temp'] >= temp_c) &
        (df['min_humidity'] <= humidity) & (df['max_humidity'] >= humidity)
    ]
    return suitable['crop'].tolist()

# STEP 4: User input
try:
    lat = float(input("Enter latitude: "))
    lon = float(input("Enter longitude: "))
    api_key = "b3e05154db77180fcbe025ce79da934c"

    # Fetch weather data
    weather = get_agro_weather(lat, lon, api_key)
    temp_c = round(weather['main']['temp'] - 273.15, 1)
    humidity = weather['main']['humidity']

    print(f"\n📍 Coordinates: ({lat}, {lon})")
    print(f"🌡️ Temperature: {temp_c}°C")
    print(f"💧 Humidity: {humidity}%")

    crops = recommend_crops(temp_c, humidity)
    if crops:
        print("\n✅ Suitable crops for these conditions:")
        for crop in crops:
            print(f" - {crop}")
    else:
        print("\n⚠️ No crops found matching the current weather conditions.")

except Exception as e:
    print("❌ Error:", e)


Enter latitude: 23.6548384
Enter longitude: 58.1838344

📍 Coordinates: (23.6548384, 58.1838344)
🌡️ Temperature: 35.0°C
💧 Humidity: 66%

⚠️ No crops found matching the current weather conditions.


In [ ]:
import requests
import pandas as pd

# Function to get city name from OpenWeatherMap
def get_city_name(lat, lon, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('name', 'Unknown Location')
    else:
        return 'Unknown Location'

# Function to get weather from Agro API
def get_agro_weather(lat, lon, api_key):
    url = f"http://api.agromonitoring.com/agro/1.0/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to get weather data")

# Function to recommend crops
def recommend_crops(temp_c, humidity):
    crops_data = [
        {"crop": "Tomato", "min_temp": 18, "max_temp": 27, "min_humidity": 50, "max_humidity": 70},
        {"crop": "Wheat", "min_temp": 10, "max_temp": 25, "min_humidity": 40, "max_humidity": 60},
        {"crop": "Rice", "min_temp": 20, "max_temp": 35, "min_humidity": 70, "max_humidity": 90},
        {"crop": "Potato", "min_temp": 15, "max_temp": 20, "min_humidity": 65, "max_humidity": 85},
        {"crop": "Lettuce", "min_temp": 10, "max_temp": 22, "min_humidity": 60, "max_humidity": 80},
        {"crop": "Onion", "min_temp": 12, "max_temp": 24, "min_humidity": 40, "max_humidity": 60},
        {"crop": "Corn", "min_temp": 16, "max_temp": 32, "min_humidity": 50, "max_humidity": 70},
        {"crop": "Carrot", "min_temp": 12, "max_temp": 23, "min_humidity": 60, "max_humidity": 80},
        {"crop": "Okra", "min_temp": 24, "max_temp": 35, "min_humidity": 50, "max_humidity": 80},
        {"crop": "Eggplant", "min_temp": 20, "max_temp": 35, "min_humidity": 60, "max_humidity": 80},
    ]
    df = pd.DataFrame(crops_data)

    perfect = df[
        (df['min_temp'] <= temp_c) & (df['max_temp'] >= temp_c) &
        (df['min_humidity'] <= humidity) & (df['max_humidity'] >= humidity)
    ]

    near_temp = df[
        (df['min_temp'] - 3 <= temp_c) & (df['max_temp'] + 3 >= temp_c)
    ]

    near_humidity = df[
        (df['min_humidity'] - 10 <= humidity) & (df['max_humidity'] + 10 >= humidity)
    ]

    return perfect, near_temp, near_humidity, df

# ==== Main Program ====
try:
    lat = float(input("Enter latitude: "))
    lon = float(input("Enter longitude: "))

    agro_api_key = "b3e05154db77180fcbe025ce79da934c"
    weather_api_key = "YOUR_OPENWEATHERMAP_KEY"  # Get your own from https://openweathermap.org/

    # Get data
    weather = get_agro_weather(lat, lon, agro_api_key)
    city_name = get_city_name(lat, lon, weather_api_key)

    temp_c = round(weather['main']['temp'] - 273.15, 1)
    humidity = weather['main']['humidity']

    print(f"\n📍 Location: {city_name} ({lat}, {lon})")
    print(f"🌡️ Temperature: {temp_c}°C")
    print(f"💧 Humidity: {humidity}%")

    perfect, near_temp, near_humidity, all_data = recommend_crops(temp_c, humidity)

    if not perfect.empty:
        print("\n✅ Perfect crop matches:")
        for crop in perfect['crop']:
            print(f" - {crop}")
    else:
        print("\n⚠️ No perfect matches found.")
        print("\n🔎 Crops with **similar temperature needs**:")
        for i, row in near_temp.iterrows():
            print(f" - {row['crop']} (Temp: {row['min_temp']}°C to {row['max_temp']}°C)")

        print("\n🔎 Crops with **similar humidity needs**:")
        for i, row in near_humidity.iterrows():
            print(f" - {row['crop']} (Humidity: {row['min_humidity']}% to {row['max_humidity']}%)")

    # Show summary
    print("\n📘 Crop Temperature & Humidity Requirements:")
    print(all_data.to_string(index=False))

except Exception as e:
    print("❌ Error:", e)
